### LSTM predict sin function

In [10]:
from tensorflow.keras.layers import Dense, LSTM, CuDNNLSTM, Dropout
from tensorflow.keras.models import Sequential
import tensorflow as tf
import numpy as np

In [21]:
TIMESTEPS = 20
TRAINING_EXAMPLES = 10000
TEST_EXAMPLES = 1000

def generate_data(seq):
    x = []
    y = []
    for i in range(len(seq) - 2*TIMESTEPS):
        x.append([seq[i:i+TIMESTEPS]])
        y.append([seq[i+TIMESTEPS:i+TIMESTEPS*2]])
        
    return np.array(x, dtype=np.float32), np.array(y, dtype=np.float32)

In [22]:
data = np.sin(np.linspace(0, 30, TRAINING_EXAMPLES + TIMESTEPS * 2, dtype=np.float32))
data.shape
len(data)
x_train, y_train = generate_data(data)

In [25]:
print(x_train.shape)
print(y_train.shape)

(10000, 1, 20)
(10000, 1, 20)


In [26]:
x_train[0, 0, :]

array([0.        , 0.00298834, 0.00597666, 0.00896492, 0.0119531 ,
       0.01494117, 0.01792911, 0.02091689, 0.02390449, 0.02689187,
       0.02987901, 0.03286588, 0.03585246, 0.03883872, 0.04182463,
       0.04481017, 0.04779531, 0.05078002, 0.05376428, 0.05674806],
      dtype=float32)

In [27]:
y_train[0, 0, :]

array([0.05973133, 0.06271408, 0.06569625, 0.06867784, 0.07165882,
       0.07463916, 0.07761883, 0.08059781, 0.08357607, 0.08655358,
       0.08953032, 0.09250626, 0.09548137, 0.09845564, 0.10142902,
       0.1044015 , 0.10737304, 0.11034363, 0.11331323, 0.11628181],
      dtype=float32)

In [41]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=x_train.shape[1:], return_sequences=True))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))

model.add(Dense(20, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)

model.compile(loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'],
             )
y_train = tf.reshape(y_train, shape=[y_train.shape[0], y_train.shape[2]])
model.fit(x_train, y_train, epochs=3, verbose=1)

ValueError: Error when checking target: expected dense_3 to have shape (1,) but got array with shape (20,)